# **Data extraction and processing**

### To view the accesses allowed for web-scraping
[Acces for web scraping (titanicfacts.net by 2024 Dave Fowler) ](https://titanicfacts.net/robots.txt)

Install the libraries:

In [34]:
# %pip install pandas numpy unidecode

In [35]:
import pandas as pd
import re
import numpy as np
from unidecode import unidecode
pd.options.mode.copy_on_write = True 
URL = 'https://titanicfacts.net/titanic-passenger-list/'
dfs = pd.read_html(URL)

In [36]:
len(dfs)

3

### Columns of interest with their descriptions.

| **Column** | **Description**                                           |
| ---------- | --------------------------------------------------------- |
| Survived   | person survived the titanic tragedy (0 - "NO", 1- "YES" ) |
| Sex        | the gender (female, male)                                 |
| Name       | the name                                                  |
| Age        | Age in years                                              |
| SibSp      | # of siblings/wife or husband on board                    |
| Parch      | # of parents / children on board                          |
| Pclass     | The ticket class (1 = "1st", 2 = "2nd", 3 = "3rd")        |

In [37]:
dfs[0].head()

,0,1,2,3,4
0,Surname,First Names,Age,Boarded,Survivor (S) or Victim (†)
1,Allen,Miss Elisabeth Walton,29,Southampton,S
2,Allison,Mr Hudson Joshua Creighton,30,Southampton,†
3,Allison,Mrs Bessie Waldo,25,Southampton,†
4,Allison,Miss Helen Loraine,2,Southampton,†


### Data extraction from the web site

In [38]:
FIRST_ROW_OF_DF=0
TICKET_CLASS_COLUMN="Pclass"
SELECTED_COLUMNS_DEL_SITE_WEB_COLUMNS=["Surname", "First Names", "Age", "Pclass"]


def set_columns_of_row(df,row):
    new_columns=df.iloc[row]
    df.columns=new_columns

def delete_row(df,row_to_drop):
    df.drop(row_to_drop,inplace=True)

def add_columns_class_ticket(df,ticket_class):
    df[TICKET_CLASS_COLUMN]=ticket_class


def get_dataframe(df,ticket_class):
    set_columns_of_row(df,FIRST_ROW_OF_DF)
    delete_row(df,FIRST_ROW_OF_DF)
    add_columns_class_ticket(df,ticket_class)
    return df[SELECTED_COLUMNS_DEL_SITE_WEB_COLUMNS]


In [39]:
dfs=[get_dataframe(df,ticket_class+1) for ticket_class,df in enumerate(dfs)]
website_df=pd.concat(dfs)
website_df.reset_index(inplace=True,drop=True)


In [40]:
website_df.head()

,Surname,First Names,Age,Pclass
0,Allen,Miss Elisabeth Walton,29,1
1,Allison,Mr Hudson Joshua Creighton,30,1
2,Allison,Mrs Bessie Waldo,25,1
3,Allison,Miss Helen Loraine,2,1
4,Allison,Master Hudson Trevor,11m,1


In [41]:
website_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1317 entries, 0 to 1316
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Surname      1317 non-null   object
 1   First Names  1317 non-null   object
 2   Age          1317 non-null   object
 3   Pclass       1317 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 41.3+ KB


### Reading csv data

In [42]:
csv_df=pd.read_csv("titanic_final_project.csv")

In [43]:
csv_df.head()

,Survived,Name,Sex,Age,SibSp,Parch,Embarked
0,0,"Braund, Mr. Owen Harris",male,22.0,1,0,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,C
2,1,"Heikkinen, Miss. Laina",female,26.0,0,0,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,S
4,0,"Allen, Mr. William Henry",male,35.0,0,0,S


In [44]:
csv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Embarked  889 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 48.9+ KB


Matching techniques:
1. Matching by surname and age, contemplating that there are no persons with the same surname and age.
2. Matching by full name using a set technique.
3. Match special cases manually.

### Data cleaning

The last name is separated from the first name of the passenger in the csv_df.

In [45]:
def extract_surname_and_first_name(full_name):
    separated_first_name = full_name.split(",")
    last_name = separated_first_name[0].strip().lower()
    first_name = separated_first_name[1].strip().lower()
    return last_name, first_name

def get_last_name_and_first_name(row):
    last_name, first_name = extract_surname_and_first_name(row["Name"])
    row["Surname"] = last_name
    row["Name"] = first_name
    return row

csv_df=csv_df.apply(get_last_name_and_first_name,axis='columns')

In [46]:
csv_df.head() 

,Survived,Name,Sex,Age,SibSp,Parch,Embarked,Surname
0,0,mr. owen harris,male,22.0,1,0,S,braund
1,1,mrs. john bradley (florence briggs thayer),female,38.0,1,0,C,cumings
2,1,miss. laina,female,26.0,0,0,S,heikkinen
3,1,mrs. jacques heath (lily may peel),female,35.0,1,0,S,futrelle
4,0,mr. william henry,male,35.0,0,0,S,allen


There are people with surnames and names with special characters in the website dataframe.

In [47]:
website_df[website_df["Surname"]=="Alhomäki"]

,Surname,First Names,Age,Pclass
623,Alhomäki,Mr Ilmari Rudolf,19,3


In [48]:
website_df[website_df["First Names"]=="Mr Far īd Husayn Qāsim"]

,Surname,First Names,Age,Pclass
621,Al-Munà,Mr Far īd Husayn Qāsim,18,3


Special characters are decoded.

In [49]:
website_df["Surname"]=website_df["Surname"].str.strip().apply(unidecode)
website_df["First Names"]=website_df["First Names"].str.strip().apply(unidecode)

The name is changed from uppercase to lowercase.

In [50]:
website_df["Surname"]=website_df["Surname"].str.lower()
website_df["First Names"]=website_df["First Names"].str.lower()

There are people with age in months and with type string in the variable age in the dataframe of the website.

In [51]:
def do_not_have_age_information(age):
    return age==""

def get_age_with_two_decimals(age):
    return round(age,2)

def age_is_in_months(age):
    return len(age)>1

def convert_age_to_years(row):
    age=re.split(r"[A-Za-z]",row["Age"])
    age_in_years=age[0]
    if do_not_have_age_information(age_in_years):
        row["Age"]=np.nan
    else:
        age_in_years = float(age_in_years)
        if age_is_in_months(age):
            age_in_years /= 12
        row["Age"] = get_age_with_two_decimals(age_in_years)
    return row

In [52]:
website_df=website_df.apply(convert_age_to_years,axis="columns")

Passengers that have the age attribute are separated, both in the CSV df and in the website df.

In [53]:
def has_age(df):
 return df["Age"].notna()

data_with_ages_not_null_csv=csv_df[has_age(csv_df)]
data_with_ages_not_null_web=website_df[has_age(website_df)]

In [54]:
data_with_ages_not_null_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  714 non-null    int64  
 1   Name      714 non-null    object 
 2   Sex       714 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     714 non-null    int64  
 5   Parch     714 non-null    int64  
 6   Embarked  712 non-null    object 
 7   Surname   714 non-null    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 50.2+ KB


In [55]:
data_with_ages_not_null_web.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1315 entries, 0 to 1316
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Surname      1315 non-null   object 
 1   First Names  1315 non-null   object 
 2   Age          1315 non-null   float64
 3   Pclass       1315 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 51.4+ KB


There are people with the same last name and the same age.

In [56]:
def has_the_same_last_name_and_age(df):
 return df[["Surname","Age"]].duplicated(keep=False)

In [57]:
data_with_ages_not_null_web[
    has_the_same_last_name_and_age(data_with_ages_not_null_web)
].sort_values(by="Age").head()

,Surname,First Names,Age,Pclass
650,asplund,master carl edgar,5.0,3
653,asplund,miss lillian gertrud,5.0,3
705,calic,mr jovo,17.0,3
704,calic,mr petar,17.0,3
197,marvin,mr daniel warner,18.0,1


In [58]:
data_with_ages_not_null_csv[
    has_the_same_last_name_and_age(data_with_ages_not_null_csv)
].sort_values(by="Age").head()

,Survived,Name,Sex,Age,SibSp,Parch,Embarked,Surname
469,1,miss. helene barbara,female,0.75,2,1,C,baclini
644,1,miss. eugenie,female,0.75,2,1,C,baclini
163,0,mr. jovo,male,17.00,0,0,S,calic
500,0,mr. petar,male,17.00,0,0,S,calic
404,0,miss. marija,female,20.00,0,0,S,oreskovic


Separamos a los que no tienen el mismo apellido y la misma edad.

In [59]:
data_with_ages_not_null_csv=data_with_ages_not_null_csv[
    ~has_the_same_last_name_and_age(data_with_ages_not_null_csv)]
data_with_ages_not_null_web=data_with_ages_not_null_web[
    ~has_the_same_last_name_and_age(data_with_ages_not_null_web)]

In [60]:
data_with_ages_not_null_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  700 non-null    int64  
 1   Name      700 non-null    object 
 2   Sex       700 non-null    object 
 3   Age       700 non-null    float64
 4   SibSp     700 non-null    int64  
 5   Parch     700 non-null    int64  
 6   Embarked  698 non-null    object 
 7   Surname   700 non-null    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 49.2+ KB


In [61]:
data_with_ages_not_null_web.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1281 entries, 0 to 1316
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Surname      1281 non-null   object 
 1   First Names  1281 non-null   object 
 2   Age          1281 non-null   float64
 3   Pclass       1281 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 50.0+ KB


### First way of matching

We do the inner join of the df of the web page with the df of the csv by the key (Last Name, Age).

In [62]:
match_first_form=pd.merge(data_with_ages_not_null_web
                          ,data_with_ages_not_null_csv
                          ,on=["Surname","Age"]
                          ,how="inner")

In [63]:
match_first_form.head()

,Surname,First Names,Age,Pclass,Survived,Name,Sex,SibSp,Parch,Embarked
0,allen,miss elisabeth walton,29.00,1,1,miss. elisabeth walton,female,0,0,S
1,allison,mrs bessie waldo,25.00,1,0,mrs. hudson j c (bessie waldo daniels),female,1,2,S
2,allison,miss helen loraine,2.00,1,0,miss. helen loraine,female,1,2,S
3,allison,master hudson trevor,0.92,1,1,master. hudson trevor,male,1,2,S
4,andrews,mr thomas,39.00,1,0,mr. thomas jr,male,0,0,S


In [64]:
match_first_form.reset_index(inplace=True,drop=True)

In [65]:
match_first_form.shape

(481, 10)

### Second way of matching

Matching based on the number of occurrences of one name within another.

First we separated those who did not have a match.

In [66]:
def get_full_name(passenger, surname_column, first_name_column):
 return passenger[surname_column].strip()+", "+passenger[first_name_column].strip()

In [67]:
surname_column="Surname"
name_column="Name"


The full name is obtained from the already matched data, since the full name will be part of our primary key.

In [68]:
match_first_form["Name"]=match_first_form.apply \
                                        (get_full_name,axis="columns"
                                        ,surname_column=surname_column
                                        ,first_name_column=name_column)

In [69]:
csv_df["Name"]=csv_df.apply(get_full_name,axis="columns"
                            ,surname_column=surname_column
                            ,first_name_column=name_column)

In [70]:
match_first_form.head()

,Surname,First Names,Age,Pclass,Survived,Name,Sex,SibSp,Parch,Embarked
0,allen,miss elisabeth walton,29.00,1,1,"allen, miss. elisabeth walton",female,0,0,S
1,allison,mrs bessie waldo,25.00,1,0,"allison, mrs. hudson j c (bessie waldo daniels)",female,1,2,S
2,allison,miss helen loraine,2.00,1,0,"allison, miss. helen loraine",female,1,2,S
3,allison,master hudson trevor,0.92,1,1,"allison, master. hudson trevor",male,1,2,S
4,andrews,mr thomas,39.00,1,0,"andrews, mr. thomas jr",male,0,0,S


In [71]:
csv_df.head()

,Survived,Name,Sex,Age,SibSp,Parch,Embarked,Surname
0,0,"braund, mr. owen harris",male,22.0,1,0,S,braund
1,1,"cumings, mrs. john bradley (florence briggs th...",female,38.0,1,0,C,cumings
2,1,"heikkinen, miss. laina",female,26.0,0,0,S,heikkinen
3,1,"futrelle, mrs. jacques heath (lily may peel)",female,35.0,1,0,S,futrelle
4,0,"allen, mr. william henry",male,35.0,0,0,S,allen


Unmatched data from the web page and the CSV are separated.

In [72]:
data_unmatched_csv=csv_df[~csv_df["Name"].isin(match_first_form["Name"])]

In [73]:
data_unmatched_csv

,Survived,Name,Sex,Age,SibSp,Parch,Embarked,Surname
1,1,"cumings, mrs. john bradley (florence briggs th...",female,38.0,1,0,C,cumings
4,0,"allen, mr. william henry",male,35.0,0,0,S,allen
5,0,"moran, mr. james",male,NaN,0,0,Q,moran
8,1,"johnson, mrs. oscar w (elisabeth vilhelmina berg)",female,27.0,0,2,S,johnson
11,1,"bonnell, miss. elizabeth",female,58.0,0,0,S,bonnell
...,...,...,...,...,...,...,...,...
875,1,"najib, miss. adele kiamie ""jane""",female,15.0,0,0,C,najib
876,0,"gustafsson, mr. alfred ossian",male,20.0,0,0,S,gustafsson
878,0,"laleff, mr. kristo",male,NaN,0,0,S,laleff
888,0,"johnston, miss. catherine helen ""carrie""",female,NaN,1,2,S,johnston


In [74]:
web_unmatched_data=website_df[website_df["Surname"].isin(data_unmatched_csv["Surname"])]

In [75]:
web_unmatched_data.head()

,Surname,First Names,Age,Pclass
0,allen,miss elisabeth walton,29.0,1
5,anderson,mr harry,47.0,1
6,andrews,miss kornelia theodosia,62.0,1
7,andrews,mr thomas,39.0,1
14,barkworth,mr algernon henry,47.0,1


Additional data is deleted on behalf of the data recovered from the website.

In [76]:
def remove_extra_info_from_name(data):
 return re.sub(r"\(.+\)",'', data)

In [77]:
names=web_unmatched_data["First Names"].apply(remove_extra_info_from_name)
web_unmatched_data.loc[:,"First Names"] = names

In [78]:
surnames=web_unmatched_data["Surname"].apply(remove_extra_info_from_name)
web_unmatched_data.loc[:,"Surname"] = surnames

The full name is extracted from the data taken from the website, which did not have a match.

In [79]:
name_column="First Names"
web_unmatched_data["Name"]=web_unmatched_data.apply \
                                              (get_full_name
                                              ,axis="columns"
                                              ,surname_column=surname_column
                                              ,first_name_column=name_column)

It is matched based on the occurrences of the full name word set of the website data present in the CSV data.

In [80]:
reg_ex= re.compile(r'[^\w\s]')
def get_if_name_is_similar(person_name_website, person_name_csv):
 person_name_website = reg_ex.sub( '', person_name_website).split()
 person_name_csv = reg_ex.sub('', person_name_csv).split()
 intersection_set=set(person_name_website).intersection(person_name_csv)
 return len(intersection_set)==len(person_name_website)

def has_same_lastname(lastname_person1,last_name_person2):
 return lastname_person1==last_name_person2

def has_similar_name(row):
 return ( has_same_lastname(row["Surname_x"],row["Surname_y"])
 and get_if_name_is_similar(row["Name_x"],row["Name_y"]) )

data_unmatched_csv['key'] = 1
web_unmatched_data['key'] = 1
matched_data_second_form = pd.merge(web_unmatched_data
                                    , data_unmatched_csv
                                    , on='key')

In [81]:
matched_data_second_form['is_a_match'] = matched_data_second_form.apply(has_similar_name
                                                            ,axis="columns")
matched_data_second_form.drop("key",axis="columns",inplace=True)

In [82]:
is_a_match=matched_data_second_form["is_a_match"]
matched_data_second_form=matched_data_second_form[is_a_match]
matched_data_second_form.reset_index(drop=True,inplace=True)

In [83]:
matched_data_second_form.shape

(252, 14)

In [84]:
matched_data_second_form.head()

,Surname_x,First Names,Age_x,Pclass,Name_x,Survived,Name_y,Sex,Age_y,SibSp,Parch,Embarked,Surname_y,is_a_match
0,anderson,mr harry,47.0,1,"anderson, mr harry",1,"anderson, mr. harry",male,48.0,0,0,S,anderson,True
1,andrews,miss kornelia theodosia,62.0,1,"andrews, miss kornelia theodosia",1,"andrews, miss. kornelia theodosia",female,63.0,1,0,S,andrews,True
2,barkworth,mr algernon henry,47.0,1,"barkworth, mr algernon henry",1,"barkworth, mr. algernon henry wilson",male,80.0,0,0,S,barkworth,True
3,baumann,mr john d.,60.0,1,"baumann, mr john d.",0,"baumann, mr. john d",male,NaN,0,0,S,baumann,True
4,beckwith,mrs sallie,46.0,1,"beckwith, mrs sallie",1,"beckwith, mrs. richard leonard (sallie monypeny)",female,47.0,1,1,S,beckwith,True


In [85]:
matched_data_second_form.rename(columns={"Surname_y":"Surname","Name_y":"Name"},inplace=True)

Some age data is in the csv and some is in the website, so we have to choose one.

In [86]:
def get_the_ages(row):
 return row["Age_y"] if not pd.isna(row["Age_y"]) else row["Age_x"]

matched_data_second_form.loc[:,"Age"]=matched_data_second_form.apply(get_the_ages ,axis="columns")

In [87]:
matched_data_second_form.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Surname_x    252 non-null    object 
 1   First Names  252 non-null    object 
 2   Age_x        250 non-null    float64
 3   Pclass       252 non-null    int64  
 4   Name_x       252 non-null    object 
 5   Survived     252 non-null    int64  
 6   Name         252 non-null    object 
 7   Sex          252 non-null    object 
 8   Age_y        139 non-null    float64
 9   SibSp        252 non-null    int64  
 10  Parch        252 non-null    int64  
 11  Embarked     252 non-null    object 
 12  Surname      252 non-null    object 
 13  is_a_match   252 non-null    bool   
 14  Age          250 non-null    float64
dtypes: bool(1), float64(3), int64(4), object(7)
memory usage: 27.9+ KB


In [88]:
match_first_form.head()

,Surname,First Names,Age,Pclass,Survived,Name,Sex,SibSp,Parch,Embarked
0,allen,miss elisabeth walton,29.00,1,1,"allen, miss. elisabeth walton",female,0,0,S
1,allison,mrs bessie waldo,25.00,1,0,"allison, mrs. hudson j c (bessie waldo daniels)",female,1,2,S
2,allison,miss helen loraine,2.00,1,0,"allison, miss. helen loraine",female,1,2,S
3,allison,master hudson trevor,0.92,1,1,"allison, master. hudson trevor",male,1,2,S
4,andrews,mr thomas,39.00,1,0,"andrews, mr. thomas jr",male,0,0,S


In [89]:
matched_data_second_form.head()

,Surname_x,First Names,Age_x,Pclass,Name_x,Survived,Name,Sex,Age_y,SibSp,Parch,Embarked,Surname,is_a_match,Age
0,anderson,mr harry,47.0,1,"anderson, mr harry",1,"anderson, mr. harry",male,48.0,0,0,S,anderson,True,48.0
1,andrews,miss kornelia theodosia,62.0,1,"andrews, miss kornelia theodosia",1,"andrews, miss. kornelia theodosia",female,63.0,1,0,S,andrews,True,63.0
2,barkworth,mr algernon henry,47.0,1,"barkworth, mr algernon henry",1,"barkworth, mr. algernon henry wilson",male,80.0,0,0,S,barkworth,True,80.0
3,baumann,mr john d.,60.0,1,"baumann, mr john d.",0,"baumann, mr. john d",male,NaN,0,0,S,baumann,True,60.0
4,beckwith,mrs sallie,46.0,1,"beckwith, mrs sallie",1,"beckwith, mrs. richard leonard (sallie monypeny)",female,47.0,1,1,S,beckwith,True,47.0


In [90]:
matched_data_second_form=matched_data_second_form[
                                match_first_form.columns]

The data from the two forms of pairing are joined.

In [91]:
matched_data=pd.concat([matched_data_second_form,match_first_form])
matched_data.dropna(subset=["Age"],inplace=True)
matched_data.reset_index(drop=True,inplace=True)

In [92]:
matched_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Surname      731 non-null    object 
 1   First Names  731 non-null    object 
 2   Age          731 non-null    float64
 3   Pclass       731 non-null    int64  
 4   Survived     731 non-null    int64  
 5   Name         731 non-null    object 
 6   Sex          731 non-null    object 
 7   SibSp        731 non-null    int64  
 8   Parch        731 non-null    int64  
 9   Embarked     730 non-null    object 
dtypes: float64(1), int64(4), object(5)
memory usage: 57.2+ KB
